# V6 Drift control

In [ ]:
import threading, time, csv
from datetime import datetime
import pyvisa
from keithley2600 import Keithley2600
import propar
import serial  # from pyserial

# --- helper to hard‐reset Keithley via SCPI+TSP ---
def reset_keithley():
    rm = pyvisa.ResourceManager("@ivi")
    inst0 = rm.open_resource(KEITHLEY_ADDR)
    inst0.timeout = 5000
    try:
        idn = inst0.query("*IDN?").strip()
        print("Keithley says:", idn)
    except Exception as e:
        print("IDN failed:", e)
        idn = ""
    # if it didn’t identify, or mis-identified:
    if "Keithley" not in idn:
        print("Resetting instrument TSP/SCPI…")
        try:
            inst0.write("*RST;*CLS;system.reset()")
        except Exception as e:
            print("Reset command error:", e)
    inst0.close()
    rm.close()
    # give it a moment to reboot
    time.sleep(15)

## Keithley

In [ ]:
# Cell 3: Thread classes with internal exception-handling & yielding
class DeviceThread(threading.Thread):
    def __init__(self, stop_evt):
        super().__init__(daemon=True)
        self.stop_evt = stop_evt
        
class KeithleyThread(DeviceThread):
    def run(self):
        fname = f"Results/{TS_STR}_resistance.csv"
        k.apply_voltage(smu, SOURCE_VOLT)
        with open(fname, "w", newline="") as f:
            w = csv.writer(f)
            w.writerow(["timestamp","voltage_V","resistance_Ohm"])
            try:
                while not self.stop_evt.is_set():
                    ts = datetime.now().isoformat()
                    try:
                        R = smu.measure.r()
                        V = smu.measure.v()
                    except Exception as e:
                        print(f"[Keithley] Error measuring: {e}")
                        self.stop_evt.set()
                        break
                    w.writerow([ts, V, R])
                    f.flush()
                    # small yield
                    time.sleep(0)
            except Exception as e:
                print("[Keithley] Thread exiting on unexpected error:", e)
                self.stop_evt.set()

In [ ]:
def ppm_to_sp(ppm: float, vessel: str = 'NO2', max_flow_rate: float = 50, total_flow_rate: float = 100) -> float:
    """
    Convert ppm to setpoint in 0..32000 range.
    ppm: float - concentration in ppm
    resolution - 0.002 ppm
    """
    vessel_dict = {'NO2': 100, 'H2S': 92}  # concentration in ppm
    mfc_correction_dict = {7: 0.5652, 8: 0.5663, 9: 0.5652, 11: 0.5660}  # correction factor for MFCs sccm per setpoint percent
    vessel_conc = vessel_dict[vessel]
    sccm = total_flow_rate * ppm / vessel_conc  # sccm
    setpoint_node7 = int(sccm / mfc_correction_dict[7] * 320 )
    setpoint_node8 = int((max_flow_rate - sccm) / mfc_correction_dict[8] * 320)
    setpoint_node9 = int(50 / mfc_correction_dict[9] * 320)
    setpoint_node11 = 0
    if (setpoint_node7 < 0 or setpoint_node7 > 32000) or (setpoint_node8 < 0 or setpoint_node8 > 32000):
        raise ValueError(f"Setpoint for node 7 or 8 is out of range: 7:{setpoint_node7}, 8:{setpoint_node8}")
    return {7: setpoint_node7, 8: setpoint_node8, 9: setpoint_node9, 11: setpoint_node11}

def segment_builder(ppm_start, ppm_end, speed, settle_time, total_flow_rate = 100, max_flow_rate = 50, vessel = 'NO2'):
    """
    Build the segments for the MFCs based on the start and end ppm, speed, settle time, total flow rate, max flow rate and vessel.
    speed: float - speed in ppm/min
    settle_time: float - settle time in seconds
    """
    # Calculate the duration of the segment in seconds
    if ppm_start == ppm_end: 
        segment_duration = settle_time
    else:  
        segment_duration = abs(ppm_end - ppm_start) / speed * 60
    segment = {'duration': segment_duration, 'ppm_start': ppm_start, 'ppm_end': ppm_end}
    return [segment]

def one_cycle(ppm_start, ppm_end, speed, settle_time, total_flow_rate = 100, max_flow_rate = 50, vessel = 'NO2'):
    """
    Build the segments for the MFCs based on the start and end ppm, speed, settle time, total flow rate, max flow rate and vessel.
    speed: float - speed in ppm/min
    settle_time: float - settle time in seconds
    """
    segments = segment_builder(ppm_start, ppm_start, speed = None, settle_time=settle_time, total_flow_rate = total_flow_rate, max_flow_rate = max_flow_rate, vessel = vessel)
    segments += segment_builder(ppm_start, ppm_end, speed = speed, settle_time=settle_time, total_flow_rate = total_flow_rate, max_flow_rate = max_flow_rate, vessel = vessel)
    segments += segment_builder(ppm_end, ppm_end, speed = None, settle_time=settle_time, total_flow_rate = total_flow_rate, max_flow_rate = max_flow_rate, vessel = vessel)
    segments += segment_builder(ppm_end, ppm_start, speed = speed, settle_time=settle_time, total_flow_rate = total_flow_rate, max_flow_rate = max_flow_rate, vessel = vessel)
    gas_usage = (sum([s['duration'] for s in segments]) * (ppm_end + ppm_start) / 2 / 60) * total_flow_rate / 100 / 1000 # in liters. 100 - NO2 vessel concentration in ppm
    return segments, gas_usage

def protocol_builder(ppm_start, ppm_end, speeds: list, speed_repeat: int, protocol_repeat: int = 1, settle_time: float = 60, total_flow_rate = 100, max_flow_rate = 50, vessel = 'NO2'):
    """speeds: list - list of speeds in ppm/min"""
    segments = []
    gas_usage = 0
    for speed in speeds:
        segment, gas_spent = one_cycle(ppm_start, ppm_end, speed, settle_time, total_flow_rate = total_flow_rate, max_flow_rate = max_flow_rate, vessel = vessel)
        segments += segment * speed_repeat
        gas_usage += gas_spent * speed_repeat
    segments *= protocol_repeat
    gas_usage *= protocol_repeat
    duration = int(sum([s['duration'] for s in segments]))
    print(f"Total time: {duration//3600} h {duration%3600//60} min \t\t Total gas usage: {gas_usage:.2f} L")
    return segments

## MFC

In [ ]:
class MFCThread(threading.Thread):
    def __init__(self,
                 node: int,
                 inst,
                 stop_evt: threading.Event,
                 logfile: str,
                 segments: list[dict],
                 vessel: str = 'NO2',
                 total_flow_rate: float = 100,
                 max_flow_rate: float = 50):
        """
        node             : MFC node address (e.g. 7 or 8)
        inst             : propar.instrument instance
        stop_evt         : Event to stop the loop
        logfile          : path to CSV output
        segments         : list of {duration, ppm_start, ppm_end}
        vessel           : gas type key into vessel_dict
        total_flow_rate  : combined flow target
        max_flow_rate    : per-MFC full-scale rate
        """
        super().__init__(daemon=True)
        self.node            = node
        self.inst            = inst
        self.stop_evt        = stop_evt
        self.logfile         = logfile
        self.segments        = segments
        self.vessel          = vessel
        self.total_flow_rate = total_flow_rate
        self.max_flow_rate   = max_flow_rate

    def run(self):
        # open CSV
        with open(self.logfile, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(['timestamp', 'set_sccm', 'meas_sccm'])
            last_flush = time.time()

            # global start time
            t0 = time.time()
            seg_idx = 0
            seg_start = t0

            # initialize first segment endpoint
            seg = self.segments[0]
            duration = seg['duration']
            ppm_start = seg['ppm_start']
            ppm_end = seg['ppm_end']

            # start the instrument
            ppm_prev = seg['ppm_start']
            setpoints = ppm_to_sp(ppm_start,vessel=self.vessel,max_flow_rate=self.max_flow_rate,total_flow_rate=self.total_flow_rate)
            self.inst.writeParameter(9, setpoints[self.node])

            # loop until all segments done or stop
            while not self.stop_evt.is_set():
                now = time.time()
                elapsed = now - t0
                seg_elapsed = now - seg_start

                # if this segment is complete, advance
                if seg_elapsed >= duration:
                    seg_idx += 1
                    if seg_idx >= len(self.segments):
                        break      # protocol complete
                    seg = self.segments[seg_idx]
                    duration    = seg['duration']
                    ppm_start   = seg['ppm_start']
                    ppm_end     = seg['ppm_end']
                    seg_start   = now
                    seg_elapsed = 0.0

                # compute instantaneous ppm
                if ppm_start == ppm_end:
                    ppm_now = ppm_start
                else:
                    ppm_now = ppm_start + (ppm_end - ppm_start) * (seg_elapsed / duration)

                # convert ppm → raw setpoint for this node
                setpoints = ppm_to_sp(ppm_now,vessel=self.vessel,max_flow_rate=self.max_flow_rate,total_flow_rate=self.total_flow_rate)
                # write + read on the same instrument
                if ppm_now != ppm_prev:
                    self.inst.writeParameter(9, setpoints[self.node])
                
                meas = self.inst.readParameter(8) / 32000 * self.max_flow_rate
                # log
                w.writerow([
                    datetime.now().isoformat(),
                    f"{(setpoints[self.node]/32000*self.max_flow_rate):.3f}",
                    f"{meas:.3f}"
                ])
                # only flush every 20 s
                if time.time() - last_flush >= 20.0:
                    f.flush()
                    last_flush = time.time()

                # tight loop: no sleep (or time.sleep(0))
                time.sleep(0.0)

        # at end: close valve
        self.inst.writeParameter(9, 0)
        self.inst.master.stop()
        print(f"[MFCThread {self.node}] protocol finished")

## params

In [ ]:
# ── User parameters ─────────────────────────────────────────────────────────
KEITHLEY_ADDR = "USB0::0x05E6::0x2601::4120420::INSTR"
SOURCE_VOLT   = 10.0               # V to source
KEITHLEY_INTEG_TIME = 0.01        # 0.001 is the smallest in-keithley delay. Increase to 0.03 for ~10Hz
# MFC params ─────────────────────────────────────────────────────────
COM_PORT      = "COM12"
MFC_NODES     = [7, 8, 9, 11]          # order: 7 → 8 → 9
MAX_FLOW      = 50.0               # sccm full-scale per MFC
TOTAL_FLOW    = 100.0              # combined flow target

In [ ]:
VESSEL        = 'NO2'
segments = protocol_builder(
    ppm_start       = 5.0,
    ppm_end         = 7.0,
    speeds          = [0.05, 0.5, 5, 15],
    speed_repeat    = 2,
    protocol_repeat = 1,
    settle_time     = 60,
    total_flow_rate = TOTAL_FLOW,
    max_flow_rate   = MAX_FLOW,
    vessel          = VESSEL
)

Total time: 3 h 14 min 		 Total gas usage: 1.16 L


## Initialization

In [ ]:
for attempt in range(2):
    try:
        k = Keithley2600(KEITHLEY_ADDR, visa_library='C:/Windows/System32/visa64.dll', raise_keithley_errors=True)
        k.connect()
        if k.connected:
            print("Keithley connected:", k.connected)
            break
        else:
            print("Resetting Keithley")
            reset_keithley()
    except Exception as e:
        print(f"Driver connect failed (attempt {attempt+1}):", e)
        if attempt == 0:
            print("-> resetting instrument and retrying…")
            reset_keithley()
        else:
            raise RuntimeError("Cannot connect to Keithley after reset") from e
k.reset()
smu = k.smua
smu.reset()
k.set_integration_time(smu, KEITHLEY_INTEG_TIME)

Keithley connected: True


In [ ]:
if COM_PORT in propar._PROPAR_MASTERS:
    master = propar._PROPAR_MASTERS[COM_PORT]
    try:
        master.stop()
    except Exception as e:
        print(e)
        pass
    del propar._PROPAR_MASTERS[COM_PORT]     # drop the old master

# now recreate
mfc = {}
setpoints = ppm_to_sp(0, VESSEL, MAX_FLOW, TOTAL_FLOW)
for node in MFC_NODES:
    inst = propar.instrument(COM_PORT, address=node)
    if not inst.master.propar.serial.is_open:
        inst.master.start()               # opens COM12 fresh
    inst.writeParameter(10,0)  # ensure that setpoint slope is off
    inst.writeParameter(9,setpoints[node])
    print(f"{COM_PORT} node {node}. ID {inst.readParameter(1)} current flow {inst.readParameter(205):.3f} sccm")
    mfc[node] = inst

COM12 node 7. ID SNM16200674A current flow 0.000 sccm
COM12 node 8. ID SNM16200674B current flow 44.018 sccm
COM12 node 9. ID SNM16200674C current flow 44.222 sccm
COM12 node 11. ID SNM16200674D current flow 0.000 sccm


## Measurements

In [ ]:
# ── Cell 4: Launch measurement threads, then wait & teardown ────────────────
TS_STR = datetime.now().strftime("%H-%M_%d-%m-%y")
stop_evt = threading.Event()
threads = []
threads.append( KeithleyThread(stop_evt) )

# 2b) One integrated MFCThread per node (control + log)
for node in [7,8]:
    inst    = mfc[node]  
    logfile = f"Results/{TS_STR}_flow_node{node}.csv"
    threads.append(
        MFCThread(
            node            = node,
            inst            = inst,
            stop_evt        = stop_evt,
            logfile         = logfile,
            segments        = segments,
            vessel          = VESSEL,
            total_flow_rate = TOTAL_FLOW,
            max_flow_rate   = MAX_FLOW))

# 3) Start them all
for t in threads:
    t.start()

# 4) Wait until all threads finish or Ctrl-C
try:
    while any(t.is_alive() for t in threads):
        time.sleep(1)
except KeyboardInterrupt:
    print("User pressed Ctrl-C; stopping…")
    stop_evt.set()
finally:
    # 5) Join threads
    for t in threads:
        t.join()

    # 6) Tear down Keithley
    try: smu.output = smu.OUTPUT_OFF
    except: pass
    try: k.disconnect()
    except: pass

    print("All threads stopped; instruments closed.")


User pressed Ctrl-C; stopping…
[MFCThread 8] protocol finished
[MFCThread 7] protocol finished
All threads stopped; instruments closed.
